# 応用情報技術者試験 類似問題ファインダー - ベクトル化 (Colab版)

このノートブックでは、複数のモデルを使用して問題文をベクトル化し、CSVファイルとして保存します。

## 対応モデル
1. **Ruri (cl-nagoya/ruri-base)**: 日本語に強いモデル
2. **Multilingual-E5 (intfloat/multilingual-e5-large)**: 多言語対応の高精度モデル

## 手順
1. 必要なライブラリをインストール
2. CSVファイルをアップロード
3. データを読み込み
4. 各モデルでベクトル化
5. 結果を保存

In [ ]:
# 1. ライブラリのインストール
%pip install sentence-transformers pandas numpy tqdm fugashi ipadic

In [ ]:
# 2. ファイルのアップロード
from google.colab import files
import os

print("CSVファイルをアップロードしてください (例: ap_siken_all_items.csv)")
uploaded = files.upload()

# アップロードされたファイル名を取得
uploaded_filename = None
for fn in uploaded.keys():
    print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
    uploaded_filename = fn
    break

In [ ]:
# 3. 設定とデータの読み込み
import pandas as pd
import numpy as np
import json
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm

# 入力ファイル (アップロードされたファイルを使用)
INPUT_FILE = uploaded_filename if uploaded_filename else "ap_siken_all_items.csv"
OUTPUT_FILE = "ap_siken_all_items_vectors.csv"

if not os.path.exists(INPUT_FILE):
    print(f"Error: {INPUT_FILE} not found. Please upload the file.")
else:
    df = pd.read_csv(INPUT_FILE, encoding='utf-8-sig')
    print(f"Data loaded: {len(df)} rows")

In [ ]:
# 4. ベクトル化関数の定義

def vectorize_sentence_transformers(model_name, texts, batch_size=32):
    print(f"Loading model: {model_name}...")
    model = SentenceTransformer(model_name)
    print(f"Vectorizing {len(texts)} texts...")
    
    vectors = model.encode(texts, batch_size=batch_size, show_progress_bar=True, convert_to_numpy=True)
    return vectors

In [ ]:
import sys
!{sys.executable} -m pip install unidic-lite
# 5. 実行

TEXT_COLUMN = "問題名"
texts = df[TEXT_COLUMN].fillna('').tolist()

# --- Model 1: Ruri ---
model_ruri = "cl-nagoya/ruri-base"
col_ruri = f"vector_{model_ruri.replace('/', '_').replace('-', '_')}"
if col_ruri not in df.columns:
    vectors_ruri = vectorize_sentence_transformers(model_ruri, texts)
    df[col_ruri] = [json.dumps(v.tolist()) for v in vectors_ruri]
    print(f"Finished {model_ruri}")

# --- Model 2: E5 ---
model_e5 = "intfloat/multilingual-e5-large"
col_e5 = f"vector_{model_e5.replace('/', '_').replace('-', '_')}"
if col_e5 not in df.columns:
    vectors_e5 = vectorize_sentence_transformers(model_e5, texts)
    df[col_e5] = [json.dumps(v.tolist()) for v in vectors_e5]
    print(f"Finished {model_e5}")

In [ ]:
# 6. 保存
df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"Saved to {OUTPUT_FILE}")

# ダウンロード (Colabの場合)
from google.colab import files
files.download(OUTPUT_FILE)